In [25]:
import os 
import json
import pandas as pd
from urllib.parse import urlparse, urlencode, parse_qs
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bs
import collections
#import downloader

api_key = "******************"

In [32]:

####순서 1
def channel_dict(fileName):
    csv_df = pd.read_csv(fileName)
    channel_dict = {}
    for i in range(len(csv_df['유튜버아이디'])):
        channel_dict[csv_df['유튜버아이디'][i]] = csv_df['유튜브링크'][i]
    return channel_dict
fileName = 'youtuber.csv'
#youtuber.csv가 같은 저장소 안에 있어야 합니다, 따로 경로 설정을 안했기때문에!
####channel_dict = channel_dict(fileName)하여 미리 만들어 놓아야 함


####순서 2
####이 함수가 정의되어야 밑에 함수가 돌아감
def get_ratings_name_date(videoId, api_key):
    
    api_data = requests.get(f"https://www.googleapis.com/youtube/v3/videos?part=statistics&id={videoId}&key={api_key}")
    json_data = json.loads(api_data.content.decode("utf-8"))
    
    api_data1 = requests.get(f"https://www.googleapis.com/youtube/v3/videos?id={videoId}&key={api_key}&part=snippet")
    json_data1 = json.loads(api_data1.content.decode("utf-8"))
    
    try:
        ratings_name_date_dict = {"title": json_data1['items'][0]['snippet']['title'],
                     "publishedAt": json_data1['items'][0]['snippet']['publishedAt'][0:10],
                     "viewCount": json_data['items'][0]['statistics']['viewCount'],
                     "likeCount": json_data['items'][0]['statistics']['likeCount'],
                     "dislikeCount": json_data['items'][0]['statistics']['dislikeCount'],
                     "commentCount": json_data['items'][0]['statistics']['commentCount']}
        return ratings_name_date_dict
    except IndexError as e: ##이 경우는, 좋아요 혹은 댓글을 막아놓은 경우인데, 이는 소통에 문제가 있다고 판단, 과감히 제꼈다.
        print(e)
        print(videoId)
        pass
    except KeyError as e:
        print(e)
        print(videoId)
        pass

####순서 3
####이 함수 정의하고, 이 함수 통해 모든 videoInfo를 갖고온다!
def channel_with_video(channel_id, api_key):
    
    api_data = requests.get(f"https://www.googleapis.com/youtube/v3/channels?part=contentDetails&id={channel_id}&maxResults=50&key={api_key}")
    json_data = json.loads(api_data.content.decode("utf-8"))
    
    play_lists = []
    for item in range(0,len(json_data['items'])):
        play_lists.append(json_data['items'][item]['contentDetails']['relatedPlaylists']['uploads'])
        
    video_list = []
    for play_list_id in play_lists:
        api_data = requests.get(f"https://www.googleapis.com/youtube/v3/playlistItems?part=snippet&playlistId={play_list_id}&maxResults=50&key={api_key}")
        json_data = json.loads(api_data.content.decode("utf-8"))
      
        for item in range(len(json_data['items'])):
            if json_data['items'][item]['snippet']['publishedAt'][:4] == '2019': #여기에 날짜 조건 추가, 지금은 2019, 나중에 변경 가능
                video_list.append(json_data['items'][item]['snippet']['resourceId']['videoId'])
        nextPageToken = json_data.get("nextPageToken")
        print(f"{len(video_list)} videos")
        i = 1
        
        while nextPageToken and i < 4:
            api_data = requests.get(f"https://www.googleapis.com/youtube/v3/playlistItems?part=snippet&playlistId={play_list_id}&pageToken={nextPageToken}&maxResults=50&key={api_key}")
            json_data = json.loads(api_data.content.decode("utf-8"))

            for item in range(len(json_data['items'])):
                if json_data['items'][item]['snippet']['publishedAt'][:4] == '2019': #여기에 날짜 조건 추가, 지금은 2019, 나중에 변경 가능
                    video_list.append(json_data['items'][item]['snippet']['resourceId']['videoId'])
                else:
                    nextPageToken = False
            
            if nextPageToken == False:
                print("{0}: {1}개 비디오 수집 완료".format(channel_id, len(video_list)))
                
            elif i == 3:
                print("{0}: {1}개 비디오 수집 완료".format(channel_id, len(video_list)))
                break
            else:
                nextPageToken = json_data.get("nextPageToken")
                i += 1
                print(f"{len(video_list)} videos")
                
    
                
    videoInfo = {}
    for Id in video_list:
        videoInfo[Id] = get_ratings_name_date(Id, api_key)
        
    channel_with_video = {channel_id: videoInfo}
    print("{0} 채널 내 모든 2019년 비디오 정보 수집 완료".format(channel_id))
    
    return channel_with_video

In [33]:
def get_subscribe(channelId, api_key):
    api_data = requests.get(f"https://www.googleapis.com/youtube/v3/channels?part=statistics&id={channelId}&key={api_key}")
    json_data = json.loads(api_data.content.decode("utf-8"))
    global data
    try:
        data = json_data["items"][0]["statistics"]["subscriberCount"]
    except:
        pass
    return data

In [28]:
channel_dict = channel_dict(fileName)

In [98]:
channel_video = []
for Id in list(channel_dict.values())[158:180]:
    channel_video.append(channel_with_video(Id, api_key))


31 videos
UCYJlyU_Lf33eqp6KWYONTGg: 31개 비디오 수집 완료
UCYJlyU_Lf33eqp6KWYONTGg 채널 내 모든 2019년 비디오 정보 수집 완료
37 videos
UCDMLdKoalJdyqKto6KahNGg: 37개 비디오 수집 완료
UCDMLdKoalJdyqKto6KahNGg 채널 내 모든 2019년 비디오 정보 수집 완료
50 videos
UCkGLGs2ZCNRSlOs9s-8S_qA: 85개 비디오 수집 완료
'commentCount'
y0inZ0a_EO0
'commentCount'
YhRFX0vgSGY
UCkGLGs2ZCNRSlOs9s-8S_qA 채널 내 모든 2019년 비디오 정보 수집 완료
29 videos
UCW61wo97HWu_TmmJEOiflrw: 29개 비디오 수집 완료
UCW61wo97HWu_TmmJEOiflrw 채널 내 모든 2019년 비디오 정보 수집 완료
42 videos
UCdpeN8LUQJlDDbRZQTWJBmA: 42개 비디오 수집 완료
UCdpeN8LUQJlDDbRZQTWJBmA 채널 내 모든 2019년 비디오 정보 수집 완료
33 videos
UCw_VHcLeiSSXA8SSk0dbwkA: 33개 비디오 수집 완료
UCw_VHcLeiSSXA8SSk0dbwkA 채널 내 모든 2019년 비디오 정보 수집 완료
28 videos
UC9edwufjEirbl9-2vns43VQ: 28개 비디오 수집 완료
UC9edwufjEirbl9-2vns43VQ 채널 내 모든 2019년 비디오 정보 수집 완료
50 videos
UCzlZ-o36GFz_hTB2br_Noow: 79개 비디오 수집 완료
UCzlZ-o36GFz_hTB2br_Noow 채널 내 모든 2019년 비디오 정보 수집 완료
36 videos
UCwjC8HF8XvKao88oJdCmJ5Q: 36개 비디오 수집 완료
UCwjC8HF8XvKao88oJdCmJ5Q 채널 내 모든 2019년 비디오 정보 수집 완료
49 videos
UCOHDZh9KGoFxmqq6t

In [100]:
with open('channel_with_video_159to180.json', 'w') as json_file:
        json.dump(channel_video, json_file)